In [1]:
import pandas as pd
from surprise import Dataset, Reader, accuracy
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV
from surprise.prediction_algorithms import knns, SVD, SVDpp, SlopeOne, NMF, BaselineOnly, NormalPredictor
from surprise.similarities import cosine, msd, pearson, pearson_baseline

In [2]:
df_rev5 = pd.read_csv('Data/df_rev5.csv')

In [3]:
df_rev5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1302158 entries, 0 to 1302157
Data columns (total 7 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   Unnamed: 0  1302158 non-null  int64 
 1   overall     1302158 non-null  int64 
 2   reviewTime  1302158 non-null  object
 3   reviewerID  1302158 non-null  object
 4   asin        1302158 non-null  object
 5   reviewText  1302158 non-null  object
 6   summary     1302158 non-null  object
dtypes: int64(2), object(5)
memory usage: 69.5+ MB


In [4]:
reader = Reader(rating_scale=(1, 5))
user_data = Dataset.load_from_df(df_rev5[['reviewerID', 'asin', 'overall']], reader)

trainset, testset = train_test_split(user_data, test_size=0.2)

In [5]:
print('Number of users: ', trainset.n_users, '\n')
print('Number of items: ', trainset.n_items, '\n')

Number of users:  97578 

Number of items:  92171 



In [6]:
baseline = NormalPredictor()
baseline.fit(trainset)
predictions = baseline.test(testset)
print(accuracy.rmse(predictions))

RMSE: 1.2320
1.2319696287657125


In [7]:
baseline2 = BaselineOnly()
baseline2.fit(trainset)
predictions = baseline2.test(testset)
print(accuracy.rmse(predictions))

Estimating biases using als...
RMSE: 0.8144
0.8143691015224391


In [ ]:
#sim_cos = {'name':'cosine', 'user_based':True}
#basic = knns.KNNBasic(k=137, sim_options=sim_cos)
#basic.fit(trainset)
#predictions = basic.test(testset)
#print(accuracy.rmse(predictions))

In [ ]:
#sim_pearson = {'name':'pearson', 'user_based':True}
#basic_pearson = knns.KNNBasic(sim_options=sim_pearson)
#basic_pearson.fit(trainset)
#predictions = basic_pearson.test(testset)
#print(accuracy.rmse(predictions))

In [ ]:
#sim_pearson = {'name':'pearson', 'user_based':False}
#basic_pearson = knns.KNNBasic(sim_options=sim_pearson)
#basic_pearson.fit(trainset)
#predictions = basic_pearson.test(testset)
#print(accuracy.rmse(predictions))

In [ ]:
#sim_pearson = {'name':'pearson', 'user_based':True}
#knn_baseline = knns.KNNBaseline(sim_options=sim_pearson)
#knn_baseline.fit(trainset)
#predictions = knn_baseline.test(testset)
#print(accuracy.rmse(predictions))

In [ ]:
#sim_pearson = {'name':'pearson', 'user_based':False}
#knn_baseline = knns.KNNBaseline(sim_options=sim_pearson)
#knn_baseline.fit(trainset)
#predictions = knn_baseline.test(testset)
#print(accuracy.rmse(predictions))

In [8]:
svd_basic = SVD(random_state=42)
results = cross_validate(svd_basic, user_data, measures=['RMSE'], cv=3, n_jobs = -1, verbose=True)

Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8136  0.8143  0.8130  0.8136  0.0005  
Fit time          46.28   46.18   46.19   46.21   0.05    
Test time         4.16    4.29    4.19    4.22    0.06    


In [10]:
results

{'test_rmse': array([0.81580095, 0.81235495, 0.81329003]),
 'fit_time': (46.68087387084961, 46.96303391456604, 46.75060248374939),
 'test_time': (4.285319566726685, 4.280826807022095, 4.181398868560791)}

In [9]:
svd_basic.fit(trainset)
predictions = svd_basic.test(testset)
print(accuracy.rmse(predictions))

RMSE: 0.8031
0.8031025653579473


In [10]:
svd_param_grid = {'n_factors':[20, 40],
                  'n_epochs': [10, 20], 
                  'lr_all': [0.002, 0.005],
                  'reg_all': [0.2 ,0.4, 0.6],
                  'biased': [True, False]}
svd_gs_model = GridSearchCV(SVD,param_grid=svd_param_grid,joblib_verbose=5,cv=3)
svd_gs_model.fit(user_data)
svd_gs_model.best_params['rmse']

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   14.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   29.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   43.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   56.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 144 out of 144 | elapsed: 49.8min finished


{'n_factors': 20,
 'n_epochs': 20,
 'lr_all': 0.005,
 'reg_all': 0.2,
 'biased': True}

In [11]:
svd_model = SVD(n_factors=20, n_epochs=20, lr_all=.005, reg_all=0.2)
svd_model.fit(trainset)
predictions = svd_model.test(testset)
print(accuracy.rmse(predictions))

RMSE: 0.7970
0.7970269869132567


In [13]:
svd_param_grid2 = {'n_factors':[5, 10, 20],
                  'n_epochs': [20, 30, 40], 
                  'lr_all': [0.5, 0.05, .005],
                  'reg_all': [0.1, 0.2]}
svd_gs2_model = GridSearchCV(SVD,param_grid=SVD_param_grid2,joblib_verbose=5, cv=3)

In [14]:
svd_gs2_model.fit(user_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   39.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   58.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  1.3min remaining:    0.0s
[Parallel(n_jobs=1)]: Done 162 out of 162 | elapsed: 83.0min finished


In [15]:
svd_gs2_model.best_params['rmse']

{'n_factors': 5, 'n_epochs': 40, 'lr_all': 0.005, 'reg_all': 0.1}

In [16]:
svd2_model = SVD(n_factors=5, n_epochs=40, lr_all=0.005, reg_all=0.1, random_state=42)
svd2_model.fit(trainset)
predictions = svd2_model.test(testset)
print(accuracy.rmse(predictions))

RMSE: 0.7818
0.7818356682247802
